In [2]:
#!pip install semantic-text-similarity
import pandas as pd
import nltk
from sklearn.ensemble import BaggingRegressor

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
#from semantic_text_similarity.models import WebBertSimilarity
#from semantic_text_similarity.models import ClinicalBertSimilarity

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from nltk.translate.bleu_score import sentence_bleu

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import random
#web_model = WebBertSimilarity(device='cpu', batch_size=10)

random.seed(42)
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.spatial import distance
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import random
random.seed(42)

In [3]:
prompts_train=pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')

In [4]:
prompts_train.prompt_text[1]

'Egyptian society was structured like a pyramid. At the top were the gods, such as Ra, Osiris, and Isis. Egyptians believed that the gods controlled the universe. Therefore, it was important to keep them happy. They could make the Nile overflow, cause famine, or even bring death. \r\nThe Egyptians also elevated some human beings to gods. Their leaders, called pharaohs, were believed to be gods in human form. They had absolute power over their subjects. After pharaohs died, huge stone pyramids were built as their tombs. Pharaohs were buried in chambers within the pyramids. \r\nBecause the people of Egypt believed that their pharaohs were gods, they entrusted their rulers with many responsibilities. Protection was at the top of the list. The pharaoh directed the army in case of a foreign threat or an internal conflict. All laws were enacted at the discretion of the pharaoh. Each farmer paid taxes in the form of grains, which were stored in the pharaoh’s warehouses. This grain was used to

In [5]:
summaries=pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')

In [6]:
data=pd.merge(summaries, prompts_train, on="prompt_id")

In [7]:
#preprocess the data




import spacy

# Data Cleaning and Preprocessing
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove special characters, numbers, and extra spaces (you can customize this)
    text = ' '.join(token for token in text.split())
    return text

# Load the spaCy English language model for tokenization and lemmatization
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def preprocess_text(text):
    # Tokenization and lemmatization using spaCy
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc ]
    return ' '.join(tokens)
def jaccard_similarity(text1, text2):
    words1 = set(text1.split())
    words2 = set(text2.split())
    intersection = len(words1.intersection(words2))
    union = len(words1) + len(words2) - intersection
    return float(intersection) / float(union)
def cosine_similarity(s1, s2):
    allsentences = [s1 , s2]
    vectorizer = CountVectorizer()
    all_sentences_to_vector = vectorizer.fit_transform(allsentences)
    text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
    text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()
    cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
    return round((1-cosine)*100,2)
def word_overlap(text1, text2):
    words1 = set(text1.split())
    words2 = set(text2.split())
    common_words = words1.intersection(words2)
    overlap_count = len(common_words)
    return overlap_count

def sentiment_diff(text1, text2):
    sentiment_score1 = analyzer.polarity_scores(text1)
    sentiment_score2 = analyzer.polarity_scores(text2)
    return abs(sentiment_score1['compound']-sentiment_score2['compound'])
def is_objective(text):
    subjective_words = ["I", "me", "my", "we", "our", "us", "you", "your", "mine", "ours"]
    
    words = text.split()
    
    for word in words:
        if word.lower() in subjective_words:
            return False  
    
    return True  


def bleu_score(original, summary):
    bleu_score = nltk.translate.bleu_score.sentence_bleu([original.split()], summary.split())
    return bleu_score

def count_stopwords(text: str) -> int:
    stopword_list = set(stopwords.words('english'))
    words = text.split()
    stopwords_count = sum(1 for word in words if word.lower() in stopword_list)
    return stopwords_count

# Count the punctuations in the text.
# punctuation_set -> !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
def count_punctuation(text: str) -> int:
    punctuation_set = set(string.punctuation)
    punctuation_count = sum(1 for char in text if char in punctuation_set)
    return punctuation_count

# Count the digits in the text.
def count_numbers(text: str) -> int:
    numbers = re.findall(r'\d+', text)
    numbers_count = len(numbers)
    return numbers_count

# This function applies all the above preprocessing functions on a text feature.
def feature_engineer(dataframe: pd.DataFrame, feature: str = 'text') -> pd.DataFrame:
    dataframe[f'{feature}_word_cnt'] = dataframe[feature].apply(lambda x: len(x.split(' ')))
    dataframe[f'{feature}_length'] = dataframe[feature].apply(lambda x: len(x))
    dataframe[f'{feature}_stopword_cnt'] = dataframe[feature].apply(lambda x: count_stopwords(x))
    dataframe[f'{feature}_punct_cnt'] = dataframe[feature].apply(lambda x: count_punctuation(x))
    dataframe[f'{feature}_number_cnt'] = dataframe[feature].apply(lambda x: count_numbers(x))
    return dataframe



# Apply data cleaning and preprocessing to the 'text' column
data['cleaned_text'] = data['text'].apply(clean_text)
data['preprocessed_text'] = data['cleaned_text'].apply(preprocess_text)
data['prompt_question'] = data['prompt_question'].apply(clean_text)
data['prompt_question'] = data['prompt_question'].apply(preprocess_text)
data['prompt_title'] = data['prompt_title'].apply(clean_text)
data['prompt_title'] = data['prompt_title'].apply(preprocess_text)
data['prompt_text'] = data['prompt_text'].apply(clean_text)
data['prompt_text'] = data['prompt_text'].apply(preprocess_text)
# lets put it in the data
data['Jaccard_Similarity'] = data.apply(lambda row: jaccard_similarity(row['prompt_text'], row['text']), axis=1)
data['cosine_similarity'] = data.apply(lambda row: cosine_similarity(row['prompt_text'], row['text']), axis=1)
data['word_overlap'] = data.apply(lambda row: word_overlap(row['prompt_text'], row['text']), axis=1)
#data['sentiment_diff'] = data.apply(lambda row: sentiment_diff(row['prompt_text'], row['text']), axis=1)

In [8]:
from nltk.corpus import stopwords
import string
import re
data=feature_engineer(data)

In [9]:
data.columns

Index(['student_id', 'prompt_id', 'text', 'content', 'wording',
       'prompt_question', 'prompt_title', 'prompt_text', 'cleaned_text',
       'preprocessed_text', 'Jaccard_Similarity', 'cosine_similarity',
       'word_overlap', 'text_word_cnt', 'text_length', 'text_stopword_cnt',
       'text_punct_cnt', 'text_number_cnt'],
      dtype='object')

In [10]:
xtrain=data[['Jaccard_Similarity','cosine_similarity','word_overlap', 'preprocessed_text', 'prompt_title','prompt_text', 'prompt_question','text_length', 'text_stopword_cnt',
       'text_punct_cnt', 'text_number_cnt']]
y_wording=data['wording']
y_content=data['content']



In [12]:
X_train_wording, X_test_wording, y_train_wording, y_test_wording = train_test_split(xtrain, y_wording, test_size=0.2, random_state=42)
X_train_content, X_test_content, y_train_content, y_test_content = train_test_split(xtrain, y_content, test_size=0.2, random_state=42)


In [13]:


# Define a ColumnTransformer to apply preprocessing to selected columns
text_vectorizer = CountVectorizer(stop_words=None)


# Define a ColumnTransformer to apply preprocessing to selected columns
preprocessor = ColumnTransformer(
    transformers=[
        ('texte', text_vectorizer, 'preprocessed_text'),
        ('title', text_vectorizer, 'prompt_title'),
        ('question', text_vectorizer, 'prompt_question'),
        ('Original', text_vectorizer, 'prompt_text')
    ], verbose_feature_names_out=False
)


In [17]:
from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
base=XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
param_grid = {
    'max_depth': [5,10,20],
    'max_features' : [5,10,20],
    'n_estimators': [100,200, 500]}

grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=0), 
param_grid=param_grid,cv=KFold(n_splits=5, shuffle=True, random_state=1))

pipeline =preprocessor.fit_transform(X_train_wording)

grid_search.fit(pipeline, y_train_wording)

GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=RandomForestRegressor(random_state=0),
             param_grid={'max_depth': [5, 10, 20], 'max_features': [5, 10, 20],
                         'n_estimators': [100, 200, 500]})

In [26]:
pipeline =preprocessor.fit_transform(X_train_wording)
model= XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)


model.fit(pipeline, y_train_wording)
pipeline1 =preprocessor.fit_transform(X_train_content)
model1= XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)

Index(['Jaccard_Similarity', 'cosine_similarity', 'word_overlap',
       'preprocessed_text', 'prompt_title', 'prompt_text', 'prompt_question',
       'preprocessed_text', 'text_length', 'text_stopword_cnt',
       'text_punct_cnt', 'text_number_cnt'],
      dtype='object')

In [44]:
from sklearn.ensemble import StackingRegressor



In [57]:



pipeline =preprocessor.fit_transform(X_train_wording)
model= BaggingRegressor(base, n_estimators=10, random_state=42)


model.fit(pipeline, y_train_wording)
pipeline1 =preprocessor.fit_transform(X_train_content)
model1= BaggingRegressor(base, n_estimators=10, random_state=42)


model1.fit(pipeline1, y_train_content)


BaggingRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                        callbacks=None, colsample_bylevel=None,
                                        colsample_bynode=None,
                                        colsample_bytree=None,
                                        early_stopping_rounds=None,
                                        enable_categorical=False,
                                        eval_metric=None, feature_types=None,
                                        gamma=None, gpu_id=None,
                                        grow_policy=None, importance_type=None,
                                        interaction_constraints=None,
                                        learning_rate=0.1, max_bin=None,
                                        max_cat_threshold=None,
                                        max_cat_to_onehot=None,
                                        max_delta_step=None, max_depth=3,
                                        max_leaves=None, min_child_weight=None,
                                        missing=nan, monotone_constraints=None,
                                        n_estimators=100, n_jobs=None,
                                        num_parallel_tree=None, predictor=None,
                                        random_state=None, ...),
                 random_state=42)

In [16]:
test_processed=preprocessor.transform(X_test_wording)
y_preds = grid_search.predict(test_processed)
mse = mean_squared_error(y_test_wording, y_preds)
mse


0.7048439593550206

In [59]:
test_processed_content=preprocessor.transform(X_test_content)
y_preds_content = model1.predict(test_processed_content)
mse1 = mean_squared_error(y_test_content, y_preds_content)
mse1

0.2538936713102005

In [60]:
summaries_test=pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
prompts_test=pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
test=pd.merge(summaries_test, prompts_test, on='prompt_id')

test['cleaned_text'] = test['text'].apply(clean_text)
test['preprocessed_text'] = test['cleaned_text'].apply(preprocess_text)
test['prompt_question'] = test['prompt_question'].apply(clean_text)
test['prompt_question'] = test['prompt_question'].apply(preprocess_text)
test['prompt_title'] = test['prompt_title'].apply(clean_text)
test['prompt_title'] = test['prompt_title'].apply(preprocess_text)
test['Jaccard_Similarity'] = test.apply(lambda row: jaccard_similarity(row['prompt_text'], row['text']), axis=1)
test['cosine_similarity'] = test.apply(lambda row: cosine_similarity(row['prompt_text'], row['text']), axis=1)
test['word_overlap'] = test.apply(lambda row: word_overlap(row['prompt_text'], row['text']), axis=1)
#data['sentiment_diff'] = data.apply(lambda row: sentiment_diff(row['prompt_text'], row['text']), axis=1)

In [61]:
xtest=preprocessor.transform(test)
y_pred = model.predict(xtest)
y_pred_content=model1.predict(xtest)

In [63]:
submission=pd.DataFrame({'student_id':test.student_id, 'content':y_pred_content, 'wording':y_pred})

In [64]:
submission.to_csv('submission.csv', index=False)

In [65]:
submission

,student_id,content,wording
0,000000ffffff,-1.198906,-1.006101
1,222222cccccc,-1.198906,-1.006101
2,111111eeeeee,-1.198906,-1.006101
3,333333dddddd,-1.198906,-1.006101
